In [260]:
# https://github.com/maxlath/wikibase-dump-filter
# https://github.com/maxlath/wikibase-dump-filter/blob/main/docs/cli.md
# extracter instance of (P31) city (Q515) eller state in US (Q35657) og som er i country (P17) USA (Q30) eller Mexico (Q96) eller Canada(Q16)
#!cat latest-all.json.gz | gzip -d | wikibase-dump-filter --languages en --claim 'P31:Q515,Q35657&P17:Q30,Q96,Q16' --omit sitelinks > city_state_america.ndjson

# extracter instance of (P31) business (Q4830453)
# led efter has subsidiary (P355)
#!cat latest-all.json.gz | gzip -d | wikibase-dump-filter --languages en --claim 'P31:Q4830453' --omit sitelinks > business.ndjson

In [261]:
# https://linuxhint.com/bash_head_tail_command/
#!head -n 1 simplified_dump.ndjson > simpl.ndjson

In [262]:
# https://stedolan.github.io/jq/tutorial/
# https://www.linode.com/docs/guides/using-jq-to-process-json-on-the-command-line/
#!jq '.' simpl.ndjson > simpl.json

In [263]:
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])
from math import ceil
import numpy as np
import pandas as pd
import requests
from SPARQLWrapper import JSON, SPARQLWrapper

from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.wikidata_query_tools import (
    retrieve_wikidata_claims,
    wikidata_query,
)

In [453]:
query_all_us_states = """
SELECT ?state
WHERE {
  ?state wdt:P31 wd:Q35657 .
}
LIMIT 10
"""
wikidata_query(sparql_query=query_all_us_states)

,state.type,state.value
0,uri,http://www.wikidata.org/entity/Q99
1,uri,http://www.wikidata.org/entity/Q173
2,uri,http://www.wikidata.org/entity/Q724
3,uri,http://www.wikidata.org/entity/Q759
4,uri,http://www.wikidata.org/entity/Q771
5,uri,http://www.wikidata.org/entity/Q779
6,uri,http://www.wikidata.org/entity/Q782
7,uri,http://www.wikidata.org/entity/Q797
8,uri,http://www.wikidata.org/entity/Q812
9,uri,http://www.wikidata.org/entity/Q816


In [454]:
def category_query(category: str):
    return f"""SELECT distinct ?item ?itemLabel ?itemDescription WHERE{{
    ?item ?label "{category}"@en .
    ?article schema:about ?item .
    ?article schema:inLanguage "en" .
    ?article schema:isPartOf <https://en.wikipedia.org/>.
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}}}"""

In [455]:
wiki_qid_of_category = wikidata_query(category_query(category="restaurant"))[
    ["item.value", "itemLabel.value", "itemDescription.value"]
].head()
wiki_qid_of_category

,item.value,itemLabel.value,itemDescription.value
0,http://www.wikidata.org/entity/Q263063,Cold Spring Tavern,restaurant
1,http://www.wikidata.org/entity/Q171475,Herberg Onder de Linden,restaurant
2,http://www.wikidata.org/entity/Q11707,restaurant,single establishment which prepares and serves...
3,http://www.wikidata.org/entity/Q1453665,Pink's Hot Dogs,restaurant
4,http://www.wikidata.org/entity/Q2068768,Heart Attack Grill,restaurant


In [267]:
def min_qid(df_qid):
    # Getting the minimum value of the QID number and the itemLabel
    index = df_qid['item.value'].apply(lambda x: int(x.split("/")[-1].replace("Q", ""))).idxmin()
    df = df_qid.loc[index][['item.value','itemLabel.value']]
    return df[0][31:], df[1]
min_qid(df_qid=wiki_qid_of_category)

('Q11707', 'restaurant')

In [325]:
test_query = wikidata_query(category_query(category="garden"))
test_query["item.value"][0][31:], test_query["itemLabel.value"][0]

('Q909444', 'Sentō Imperial Palace')

In [328]:
df = (wikidata_query(category_query(category="garden"))[['itemLabel.value', 'item.value']])
df

,itemLabel.value,item.value
0,Sentō Imperial Palace,http://www.wikidata.org/entity/Q909444
1,garden,http://www.wikidata.org/entity/Q1107656
2,Soswaewon,http://www.wikidata.org/entity/Q493693
3,Parco del Valentino,http://www.wikidata.org/entity/Q1061804
4,Classical Gardens of Suzhou,http://www.wikidata.org/entity/Q1144337
5,McGovern Centennial Gardens,http://www.wikidata.org/entity/Q21196493
6,Schedel Arboretum and Gardens,http://www.wikidata.org/entity/Q7431032
7,Gardens of the French Renaissance,http://www.wikidata.org/entity/Q5522491
8,Désert de Retz,http://www.wikidata.org/entity/Q1270698
9,Weymouth Peace Garden,http://www.wikidata.org/entity/Q28003099


In [326]:
min_qid(wikidata_query(category_query(category="garden")))

('Q493693', 'Soswaewon')

In [271]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

# schema_types = pd.read_csv(get_path("schemaorg-current-https-types.csv"))

In [272]:
categories_unique = list(set(biz["categories"].str.cat(sep=", ").split(sep=", ")))
categories = list(biz["categories"].str.cat(sep=", ").split(sep=", "))

In [273]:
from collections import Counter

category_occurences = pd.DataFrame(list(dict(Counter(categories)).items()), columns=["category", "occurences"]).sort_values(by="occurences", ascending=False)

In [274]:
display(category_occurences)

,category,occurences
17,Restaurants,52268
18,Food,27781
12,Shopping,24395
135,Home Services,14356
107,Beauty & Spas,14292
...,...,...
1261,Beach Bars,1
1267,DUI Schools,1
1270,Patent Law,1
1274,Housing Cooperatives,1


In [275]:
word = "Undersea/Hyperbaric Medicine"


def split_word_inc_slash(word):
    # Splitting the words that have a slash in them, and turning them into two words
    word_space = word.split(" ")
    word_space
    new_wordlist_a = []
    new_wordlist_b = []
    for i in word_space:
        i = i.lower()
        if "/" not in i:
            new_wordlist_a.append(i)
            new_wordlist_b.append(i)
        else:
            slash_split = i.split("/")
            new_wordlist_a.append(slash_split[0])
            new_wordlist_b.append(slash_split[1])
    new_word_a = " ".join(new_wordlist_a)
    new_word_b = " ".join(new_wordlist_b)
    return [new_word_a, new_word_b]


split_word_inc_slash(word)

['undersea medicine', 'hyperbaric medicine']

In [276]:
# transforms Yelp category words to words that is succesfull in finding the QID on wikidata
# TODO: fix & split issues
# TODO: fix & split has to be done first, and / split after
import re
import inflect
p = inflect.engine()

categories_dict = {}
for word in categories_unique:
    if '&' in word and '/' in word:
        word_list = re.split('&|/',word.lower())
        print(word_list)
        categories_dict[word] = word_list
    elif '&' in word:
        word_list = list(filter(None, word.lower().split(sep=' & ')))
        categories_dict[word] = word_list
    elif '/' in word:
        categories_dict[word] = split_word_inc_slash(word)
    else:
        categories_dict[word] = [word.lower()]

categories_dict_singular = {}
for key, value in categories_dict.items():
    new_value = []
    for word in value:
        if p.singular_noun(word) is False:
            word = word
        else:
            word = p.singular_noun(word)
        new_value.append(word)
    categories_dict_singular[key] = new_value

['heating ', ' air conditioning', 'hvac']


In [277]:
categories_dict_singular['Heating & Air Conditioning/HVAC']

['heating ', ' air conditioning', 'hvac']

In [278]:
category_occurences["split_category"] = category_occurences["category"].map(
    categories_dict_singular
)
category_occurences = category_occurences.explode("split_category")

In [279]:
category_occurences[category_occurences["category"].str.contains("/")][
    ["category", "occurences"]
].drop_duplicates()

,category,occurences
436,Heating & Air Conditioning/HVAC,1199
189,Cajun/Creole,923
610,Water Heater Installation/Repair,557
396,Community Service/Non-Profit,503
363,Blow Dry/Out Services,456
331,Tapas/Small Plates,440
177,Masonry/Concrete,363
218,Door Sales/Installation,277
348,Bike Repair/Maintenance,273
2,Naturopathic/Holistic,221


In [546]:
andlist = category_occurences[category_occurences["category"].str.contains("&")]["category"].drop_duplicates().to_list()

In [392]:
category_occurences[category_occurences["category"].str.contains("air")]

,category,occurences,split_category,qid,qid_label
223,Auto Repair,5433,auto repair,NaN,None
105,Hair Salons,5046,hair salon,NaN,None
116,Hair Removal,3239,hair removal,Q625145,depilation
281,Hair Stylists,1459,hair stylist,Q55187,hairdresser
96,IT Services & Computer Repair,1189,it service,NaN,None
96,IT Services & Computer Repair,1189,computer repair,NaN,None
365,Men's Hair Salons,828,men's hair salon,NaN,None
152,Appliances & Repair,800,appliance,Q212920,home appliance
152,Appliances & Repair,800,repair,NaN,None
464,Transmission Repair,639,transmission repair,NaN,None


In [497]:
category_qid = {}
category_qid2 = {}
for cat in category_occurences["split_category"].to_list():
    try:
        wikidata_query_cat_query = wikidata_query(category_query(category=cat)) # Querys wikidata for the QID of the category
        category_qid[cat] = (wikidata_query_cat_query["item.value"][0][31:],wikidata_query_cat_query["itemLabel.value"][0]) # Adds QID and label of the first result of the query
        category_qid2[cat] = min_qid(wikidata_query_cat_query) # Adds QID and label with min_qid function
    except:
        pass


In [500]:
from pprint import pprint
from deepdiff import DeepDiff
category_qid_only_qid = {key:value for (key,value) in category_qid.items()}
category_qid2_only_qid = {key:value for (key,value) in category_qid2.items()}
# compares the two dictionaries and returns the differences in old value and new value for every key
ddiff = DeepDiff(category_qid_only_qid, category_qid2_only_qid, verbose_level=1) 

In [505]:
ddiff

{'values_changed': {"root['restaurant'][0]": {'new_value': 'Q11707',
   'old_value': 'Q1453665'},
  "root['restaurant'][1]": {'new_value': 'restaurant',
   'old_value': "Pink's Hot Dogs"},
  "root['food'][0]": {'new_value': 'Q2095', 'old_value': 'Q894875'},
  "root['food'][1]": {'new_value': 'food', 'old_value': 'Botargo'},
  "root['bar'][0]": {'new_value': 'Q860', 'old_value': 'Q28337'},
  "root['bar'][1]": {'new_value': 'foobar', 'old_value': 'shoal'},
  "root['automotive'][0]": {'new_value': 'Q1850259', 'old_value': 'Q7981858'},
  "root['automotive'][1]": {'new_value': 'Renault Rodeo',
   'old_value': 'Welsh Automotive Forum'},
  "root['service'][0]": {'new_value': 'Q44127', 'old_value': 'Q906066'},
  "root['service'][1]": {'new_value': 'server', 'old_value': 'church service'},
  "root['sandwich'][0]": {'new_value': 'Q28803', 'old_value': 'Q7689067'},
  "root['sandwich'][1]": {'new_value': 'sandwich',
   'old_value': 'Tavern sandwich'},
  "root['fast food'][0]": {'new_value': 'Q8179

In [523]:
tuple_test = ("Q123", "test")
tuple_update = list(tuple_test)
tuple_update[0] = "Q456"
tuple_update[1] = "test2"
tuple(tuple_update)

('Q456', 'test2')

In [527]:
dict = {'airline': 'Q46970', 'boat tour': 'Q25040412', 'magazine': 'Q41298'}
update_dict = {'root["airline"][0]': 'Q8782', 'root["boat tour"][0]': 'Q981249', 'root["magazine"][0]': 'Q25738'}
for key, value in ddiff['values_changed'].items():
    if key.__contains__("[0]") is True:
        print(key[6:-5], )

restaurant Q1453665 ('Q1453665', "Pink's Hot Dogs")
food Q894875 ('Q894875', 'Botargo')
bar Q28337 ('Q28337', 'shoal')
automotive Q7981858 ('Q7981858', 'Welsh Automotive Forum')
service Q906066 ('Q906066', 'church service')
sandwich Q7689067 ('Q7689067', 'Tavern sandwich')
fast food Q25234965 ('Q25234965', 'Knysza')
home Q3046346 ('Q3046346', 'home')
garden Q909444 ('Q909444', 'Sentō Imperial Palace')
burger Q588537 ('Q588537', 'Anton Burger')
entertainment Q1501342 ('Q1501342', 'Vent')
instruction Q1057179 ('Q1057179', 'user guide')
dentist Q4933526 ('Q4933526', 'Bob Norton')
venue Q1154727 ('Q1154727', 'Takemoto-za')
gift Q707482 ('Q707482', 'gift')
convenience store Q28405997 ('Q28405997', 'All Day Convenience Store')
gym Q5099261 ('Q5099261', 'Chilwell Olympia')
pub Q106652262 ('Q106652262', 'The Coat & Badge, Putney')
accessory Q1065579 ('Q1065579', 'costume accessory')
cosmetic Q8243409 ('Q8243409', 'eyeliner')
hair stylist Q16734435 ('Q16734435', 'Peter Coppola')
veterinarian Q6

In [531]:
def compare_qids(new_value: str, old_value: str):
    # check if the new qid is an instance of old qid
    return f"""SELECT ?s 
                WHERE {{?s wdt:P31 wd:{old_value} . 
                        VALUES ?s {{wd:{new_value}}} .
                }}"""

update_qid_dict = {}
for key, value in ddiff['values_changed'].items():
    if key.__contains__("[0]") is True:
        # check if the new qid is an instance of old qid, then update with old qid if true
        if wikidata_query(compare_qids(new_value=value['new_value'], old_value=value['old_value'])).empty is False:
            print(f"Updating {key} from {value['new_value']} to {value['old_value']}")
            update_qid_dict[key[6:-5]] = category_qid[key[6:-5]]
category_qid2.update(update_qid_dict) # update the qid dict with the new qids, updated values: {'airline': 'Q46970', 'boat tour': 'Q25040412', 'magazine': 'Q41298'}

Updating root['airline'][0] from Q8782 to Q46970
Updating root['boat tour'][0] from Q981249 to Q25040412
Updating root['magazine'][0] from Q25738 to Q41298


In [494]:
category_qid2['airline']

'Q46970'

In [315]:
category_occurences['qid'] = category_occurences['split_category'].map(category_qid2)
category_occurences[['qid','qid_label']] = pd.DataFrame(category_occurences['qid'].tolist(),index=category_occurences.index)

In [317]:
category_qid_list = category_occurences["qid"].tolist()
category_qid_list = [i for i in category_qid_list if i is not np.nan]

In [318]:
item_list = category_qid_list

item_list_len = len(item_list)
# The limit is set to meet the requirements of the wikibase API wbgetentities (max 50)
# Ceil makes sure that the each subset from item_list is no longer than 50
limit = ceil(item_list_len / 50)

# Seperates the item_list to a nested_list with max 50 items in each list
piped_list = [item_list[pipe::limit] for pipe in range(limit)]

category_wikidata = {}
for i in piped_list:
    category_wikidata.update(retrieve_wikidata_claims(i))

In [319]:
category_triple = {}
for key, values in category_wikidata.items():
    for value in values:
        for obj in value:
            if obj["mainsnak"]["property"] == "P279":
                data_value = obj["mainsnak"]["datavalue"]["value"]["id"]
                category_triple[key] = category_triple.get(key, []) + [data_value]
category_triple

{'Q11707': ['Q62602544', 'Q1431026', 'Q41176', 'Q41958'],
 'Q6097': ['Q61951',
  'Q19359564',
  'Q1365365',
  'Q11090700',
  'Q2647467',
  'Q473666'],
 'Q39201': ['Q622852'],
 'Q1294114': ['Q999981'],
 'Q188507': ['Q699405', 'Q1497384'],
 'Q837171': ['Q7406919', 'Q25351891', 'Q2424752', 'Q806750', 'Q815823'],
 'Q1996635': ['Q5422651', 'Q213441', 'Q108290424'],
 'Q124946': ['Q48803', 'Q47728'],
 'Q181055': ['Q746549', 'Q2957687', 'Q28803', 'Q81799', 'Q7427595'],
 'Q211578': ['Q3632343', 'Q2207288', 'Q17200001'],
 'Q1318959': ['Q213441'],
 'Q1971625': ['Q206615'],
 'Q364005': ['Q13226383', 'Q121359', 'Q178706'],
 'Q22657': ['Q181790', 'Q34669510'],
 'Q1880871': ['Q18643213'],
 'Q1501': ['Q1076486', 'Q12004466'],
 'Q317309': ['Q1403186', 'Q19829125', 'Q12147'],
 'Q1592332': ['Q1374250'],
 'Q8341': ['Q373342'],
 'Q11033': ['Q340169', 'Q17537576', 'Q121182', 'Q24229398'],
 'Q339836': ['Q3897491'],
 'Q39809': ['Q8187769', 'Q42240'],
 'Q3095365': ['Q148571'],
 'Q1410837': ['Q37038'],
 'Q84684

In [320]:
wiki_subclasses = pd.DataFrame(
    list(category_triple.items()), columns=["category_qid", "subclassOf"]
).explode("subclassOf")

In [321]:
wiki_subclasses

,category_qid,subclassOf
0,Q11707,Q62602544
0,Q11707,Q1431026
0,Q11707,Q41176
0,Q11707,Q41958
1,Q6097,Q61951
...,...,...
584,Q895060,Q1368898
584,Q895060,Q15855160
584,Q895060,Q877517
585,Q942297,Q362200


In [381]:
df = category_occurences.merge(
    wiki_subclasses, left_on="qid", right_on="category_qid", how="left"
)

In [441]:
schema_mappings = pd.read_csv(get_path("class_mappings.csv"))
df['split_category'] = df['split_category'].apply(lambda x: x.title().replace(' ',''))
df1 = df.merge(schema_mappings, left_on="split_category", right_on="YelpCategory", how="left")


In [451]:
df1.loc[df1['qid'] == 'Q103597']

,category,occurences,split_category,qid,qid_label,category_qid,subclassOf,YelpCategory,SchemaType
900,Surgeons,123,Surgeon,Q103597,Georg Hartog Gerson,NaN,NaN,NaN,NaN


In [452]:
from rdflib import Namespace, Graph, URIRef, Literal, BNode
from rdflib.namespace import RDFS
import urllib.parse
schema = Namespace("https://schema.org/")
example = Namespace("https://example.org/")
wiki = Namespace("https://www.wikidata.org/entity/")
# triple_file = gzip.open(filename=f"/home/ubuntu/vol1/virtuoso/import/yelp_business.nt.gz", mode="at",encoding="utf-8")

G = Graph()
for i in df1.itertuples():
    if i.subclassOf is not np.nan:
        G.add((wiki[i.qid], wiki["P279"], wiki[i.subclassOf]))
    if i.qid is not np.nan:
        G.add((wiki[i.qid], RDFS["label"], Literal(i.qid_label)))
        G.add((wiki[i.qid], RDFS["Class"], example['WikiCategory']))
        if i.SchemaType is not np.nan:
            G.add((schema[i.SchemaType], schema["sameAs"], wiki[i.qid]))
        else:
            G.add((example[i.split_category], schema["sameAs"], wiki[i.qid]))

nt = G.serialize(destination="categories.nt", format="nt")


# triple_file.write(G.serialize(format="nt"))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/rdflib/plugins/serializers/nt.py:35: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


'coffee and tea' superclassOf 'qid of coffee'
'coffee and tea' superclassOf 'qid of tea'

'Restaurants' schema:sameAs 'qid of restaurant'

'qid of coffee' wiki:subclassOf(P279) 'qid'
'qid of tea' wiki:subclassOf(P279) 'qid'

schema:'restaurant' schema:sameAs wiki:'restaurant'